# Writing New Code with Copilot

We can use Copilot to write new pieces of code. When doing so, the following are useful tips:

1. **Provide a description**: Copilot can generate code based on the description you provide. The more detailed the description, the better the code generated.
2. **Be specific**: For example, when writing a function, you might specify:
    - The types and purposes of arguments
    - The return type
    - The expected behavior
    - Any constraints or requirements
    - If and when exceptions should be raised
3. **Iterate**: Copilot may not get it right the first time. Describe problems and what you would like done differently. This can be done in chat, or by make iterative changes in the editor.
4. **Set the style**: Copilot is aware of other code in the file and can generate code that is consistent with it. Writing some code before asking Copilot to generate more can help it understand the style you are looking for.
5. **Be critical of suggestions**: Copilot may generate code that is not correct or not what you want. It is important to review the code and make sure it is correct and does what you want.
6. **Test the code**: You should always test code you write, preferably by writing formal tests in a unit-testing framework such as ```unittest``` or ```pytest``` in Python. This is doubly important when using Copilot, as the code may not be correct or may not do what you expect. Copilot can help you write tests.
7. **Understand the code**: Copilot cannot replace the need for you to understand the code you're writing. It is best suited to speeding up the production of tasks you could do yourself. It may be able to help you produce code one step more complicated than you could do yourself, but you should make sure you understand what is going on before you use it. This can be a good way to learn new techniques or patterns, and can be aided by asking Copilot for explanations of bits you don't understand. If you try to produce code that is substantially beyond your current abilities, you will likely make mistakes and not understand how to fix them. As a result, you should carry on developing your skills as a programmer instead of relying on Copilot (or any other AI tool) to do the work for you.
8. **Break down large tasks**: Copilot is better at generating small pieces of code than large ones. If you have a large task, try to break it down into smaller pieces that you can ask Copilot to generate for you. This will make it easier for Copilot to understand what you want and generate the correct code. Copilot can help you with this process by suggesting ways to break down the task.


## Example

In the code cell below, the tutor will demonstrate how to write a function which accepts a number of data series in a Numpy array, and produces a line plot in Matplotlib.

## Exercise

Set yourself the task of writing a function that you could write yourself in 30-60 minutes. See if you can get Copilot to generate the code for you. While doing this be mindful of the following questions:
- Be mindful of what you are asking Copilot to do. What worked well? What didn't?
- Is the code correct?
- Is it different from what you would have written? Is it better of worse?
- Did you learn any new programming techniques?


In [18]:
import random

def generate_maze(size):
    maze = [['#' for _ in range(size)] for _ in range(size)]
    start = (0, 0)
    end = (size - 1, size - 1)
    maze[start[0]][start[1]] = 'S'
    maze[end[0]][end[1]] = 'E'

    def carve_passages_from(cx, cy):
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        random.shuffle(directions)
        for direction in directions:
            nx, ny = cx + direction[0], cy + direction[1]
            if 0 <= nx < size and 0 <= ny < size and maze[nx][ny] == '#':
                if sum(1 for dx, dy in directions if 0 <= nx + dx < size and 0 <= ny + dy < size and maze[nx + dx][ny + dy] == ' ') < 2:
                    maze[nx][ny] = ' '
                    carve_passages_from(nx, ny)

    maze[1][0] = ' '  # Ensure start is connected
    carve_passages_from(1, 0)
    return maze

def solve_maze(maze):
    # Directions: (row_offset, col_offset)
    directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]  # Up, Right, Down, Left
    direction = 1  # Start by facing right
    row, col = 0, 0  # Start position

    def is_within_bounds(r, c):
        return 0 <= r < len(maze) and 0 <= c < len(maze[0])

    def is_open(r, c):
        return maze[r][c] in [' ', 'E']

    path = [(row, col)]

    while maze[row][col] != 'E':
        # Try to turn left
        new_direction = (direction - 1) % 4
        new_row, new_col = row + directions[new_direction][0], col + directions[new_direction][1]

        if is_within_bounds(new_row, new_col) and is_open(new_row, new_col):
            direction = new_direction
            row, col = new_row, new_col
        else:
            # Try to move forward
            new_row, new_col = row + directions[direction][0], col + directions[direction][1]
            if is_within_bounds(new_row, new_col) and is_open(new_row, new_col):
                row, col = new_row, new_col
            else:
                # Turn right
                direction = (direction + 1) % 4

        path.append((row, col))

    return path

# Generate a 10x10 maze
maze = generate_maze(10)

# Solve the maze
solution_path = solve_maze(maze)

# Print the maze and the solution path
for r in range(len(maze)):
    for c in range(len(maze[r])):
        if (r, c) in solution_path:
            print('.', end=' ')
        else:
            print(maze[r][c], end=' ')
    print()

. # . . . . # . . # 
. . . # # . . # . . 
# # #     # . . # . 
      #     # . . . 
  #     #     # # . 
    #       #     . 
  #     #     # # . 
  #   #   # #     . 
  #             # . 
    #   #   #     . 


In [19]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib.colors as mcolors
from IPython.display import HTML

def animate_maze_solution(maze, path):
    """
    Create an animation showing the path through a maze
    
    Parameters:
    maze (list): 2D list representing the maze where '#' are walls and '.' are paths
    path (list): List of (row, col) tuples representing the solution path
    
    Returns:
    HTML: Animation that can be displayed in a Jupyter notebook
    """
    # Convert maze to numpy array
    maze_array = np.array([[1 if cell == '#' else 0 for cell in row] for row in maze])
    
    # Setup the figure
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Create custom colormap: white=path, gray=wall, red=visited, green=current
    colors = ['white', 'gray', 'red', 'green']
    cmap = mcolors.ListedColormap(colors)
    
    def update(frame):
        ax.clear()
        current = maze_array.copy()
        
        # Show path up to current position
        for i in range(min(frame, len(path))):
            current[path[i][0], path[i][1]] = 2  # visited
        
        # Show current position
        if frame < len(path):
            current[path[frame][0], path[frame][1]] = 3  # current
            
        # Plot the maze
        ax.imshow(current, cmap=cmap)
        ax.grid(True)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(f'Maze Solution - Step {frame}')
    
    # Create animation
    anim = FuncAnimation(fig, update, 
                        frames=len(path) + 1,
                        interval=200,  # 200ms between frames
                        repeat=False)
    
    # Close the figure to prevent display
    plt.close()
    
    # Return HTML animation
    return HTML(anim.to_jshtml())

# Example usage in your notebook:
# animation = animate_maze_solution(maze, solution_path)
# display(animation)

In [21]:
# Print the maze and the solution path
for r in range(len(maze)):
    for c in range(len(maze[r])):
        if (r, c) in solution_path:
            print('.', end=' ')
        else:
            print(maze[r][c], end=' ')
    print()

. # . . . . # . . # 
. . . # # . . # . . 
# # #     # . . # . 
      #     # . . . 
  #     #     # # . 
    #       #     . 
  #     #     # # . 
  #   #   # #     . 
  #             # . 
    #   #   #     . 


In [22]:
# Assuming maze and solution_path are already defined
animation = animate_maze_solution(maze, solution_path)
display(animation)